In [63]:
import pandas as pd


# I Importation des données

In [64]:
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")
X_test = pd.read_csv("data/X_test.csv")
y_test = pd.read_csv("data/y_test.csv")

# Pipeline et modèle

Il ne faut selectionner que 10 features, on supprime Neighborhood qui est trop complexe à traiter, ainsi que bsmt Qual et garage Finish qui sont trop proches d'autres variables

## Transformation du data set

In [65]:
numeric_features = ["Year Built", "Total Bsmt SF", "1st Flr SF", "Gr Liv Area", "Garage Area", "Overall Qual", "Full Bath"]
ordinal_features = [ "Exter Qual",  "Kitchen Qual"]
cat_feature = ["Neighborhood"]
all_col = numeric_features.copy()
all_col.extend(ordinal_features)
all_col.extend(cat_feature)



In [76]:
def transformation_X(X):
       X = X[all_col]
       X.columns = ['Year_Built', 'Total_Bsmt_SF', '1st_Flr_SF', 'Gr_Liv_Area','Garage_Area', 'Overall_Qual', 'Full_Bath', 'Exter_Qual',
              'Kitchen_Qual', 'Neighborhood']
       return X

In [67]:
X_train = transformation_X(X_train)
X_test = transformation_X(X_test)

## Modèle

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OrdinalEncoder

numeric_transformer = SimpleImputer()

exter_cat = [ 'Po', 'Fa','TA', 'Gd','Ex']
kitchen_cat = [ 'Po', 'Fa','TA', 'Gd',"Ex"]

ordinal_transformer = OrdinalEncoder(categories=[exter_cat, kitchen_cat])

categorical_transformer = OneHotEncoder()



In [70]:
numeric_features = ["Year_Built", "Total_Bsmt_SF", "1st_Flr_SF", "Gr_Liv_Area", "Garage_Area", "Overall_Qual", "Full_Bath"]
ordinal_features = [ "Exter_Qual",  "Kitchen_Qual"]
cat_feature = ["Neighborhood"]

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('cat', categorical_transformer, cat_feature)
    ]
)

In [71]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [72]:
import numpy as np
from sklearn.metrics import mean_absolute_error

pipe = Pipeline([
     ('preprocessor', preprocessor),
     ('reg', reg)
])


pipe.fit(X_train, y_train) 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Year_Built',
                                                   'Total_Bsmt_SF',
                                                   '1st_Flr_SF', 'Gr_Liv_Area',
                                                   'Garage_Area',
                                                   'Overall_Qual',
                                                   'Full_Bath']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex'],
                                                                             ['Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex']]),
                                                  ['Exter_Qual',
                                                   'Kitchen_Qual']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Neighborhood'])])),
                ('reg', LinearRegression())])

In [73]:
X_test

,Year_Built,Total_Bsmt_SF,1st_Flr_SF,Gr_Liv_Area,Garage_Area,Overall_Qual,Full_Bath,Exter_Qual,Kitchen_Qual,Neighborhood
0,2000,924.0,948,1690,463.0,6,2,TA,TA,Gilbert
1,2007,1482.0,1482,2574,868.0,8,2,Gd,Ex,CollgCr
2,2001,1568.0,1568,1568,564.0,8,2,Gd,Gd,Somerst
3,1969,1248.0,1338,2634,968.0,5,2,TA,TA,NAmes
4,1947,0.0,672,912,0.0,4,1,TA,TA,BrkSide
...,...,...,...,...,...,...,...,...,...,...
581,1939,856.0,856,856,399.0,6,1,TA,TA,SWISU
582,1971,864.0,864,864,0.0,4,1,TA,TA,NAmes
583,1950,768.0,768,1328,308.0,5,1,TA,TA,OldTown
584,2003,1682.0,1700,1700,544.0,6,2,Gd,Gd,CollgCr


In [74]:
from sklearn.metrics import mean_absolute_error
pipe.score(X_test,y_test)

predict_train  = pipe.predict(X_train)
predict_test  = pipe.predict(X_test)

# Root Mean Squared Error on train and test date
print('MAE on train data: ', mean_absolute_error(y_train, predict_train))
print('MAE on test data: ',  mean_absolute_error(y_test, predict_test))



MAE on train data:  10873.597604498133
MAE on test data:  10051.715478411643


pipe['reg'].get_feature_names_out

In [ ]:
# import pickle
# filename = 'finalized_model.sav'
# pickle.dump(search, open(filename, 'wb'))